In [5]:
import numpy as np
import pandas as pd
import os
import re
from glob import glob

In [6]:
G17_raw_data = sorted(glob('./raw_data/G17/G*.txt'))

In [7]:
G17_raw_data_name = list(map(lambda x:x[15:],G17_raw_data))

In [8]:
i = 0
all_data = pd.DataFrame()
for file in G17_raw_data:
    data = pd.read_csv(file, sep = '\t')
    data['data_name'] = G17_raw_data_name[i]
    data.columns = data.columns.str.replace('  ','')
    data.columns = data.columns.str.replace('G17 -','G17-') #有些奇怪的間隔
    data['data_point'] = data.index
    all_data = pd.concat([data,all_data])
    all_data = all_data.drop([0]) #消除單位Deg.
    all_data.reset_index(drop = True)
    i = i + 1

C:\Users\AutoUser\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [9]:
all_data.columns

Index(['G17-PTC10', 'G17-PTC11', 'G17-PTC12', 'G17-PTC13', 'G17-PTC14',
       'G17-PTC15', 'G17-PTC16', 'G17-PTC17', 'G17-PTC18', 'G17-PTC5',
       'G17-PTC7', 'G17-PTC8', 'G17-PTC9', 'Unnamed: 5', 'Unnamed: 8',
       'data_name', 'data_point'],
      dtype='object')

In [10]:
#drop 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8' 的columns
#rows出現 Deg的資料 欄位
for column in all_data.columns:
    if column[0] == 'U':
        all_data.drop([column],axis = 1,inplace = True)

In [11]:
all_data['data_name'] = all_data['data_name'].apply(lambda x:x[:-11]) #去掉 _Export.txt

In [12]:
all_data['data_name'].tail()

314    G17-AC(7X15)20160114-001
315    G17-AC(7X15)20160114-001
316    G17-AC(7X15)20160114-001
317    G17-AC(7X15)20160114-001
318    G17-AC(7X15)20160114-001
Name: data_name, dtype: object

In [13]:
all_data['recipes'] = all_data['data_name'].str.extract(r'(G\d+)',expand = False) #抓出所屬的Recipes
all_data['autoclave'] = all_data['data_name'].str.extract(r'(AC\(\d+x\d+\))',expand = False,flags=re.IGNORECASE) #用正則抓出 所屬的熱壓爐
all_data['date'] = all_data['data_name'].str.extract(r'(\d{7,8})',expand = False) #抓出日期
all_data['number'] = all_data['data_name'].str.extract(r'(00\d{1,3})',expand = False)#抓出所屬編號
all_data.drop(['data_name'],axis = 1,inplace = True)

In [14]:
all_data.columns = all_data.columns.str.replace('G17-','')

In [15]:
PTC_list = []
for column in all_data.columns:
    if column[0] == 'P' and column[-1] != 't':
        PTC_list.append(column)

In [16]:
all_data['PTC_count'] = len(PTC_list) - (all_data.shape[1] - all_data.count(axis = 1)) 
#每筆資料所收集溫度參數的個數

In [17]:
all_data.columns

Index(['PTC10', 'PTC11', 'PTC12', 'PTC13', 'PTC14', 'PTC15', 'PTC16', 'PTC17',
       'PTC18', 'PTC5', 'PTC7', 'PTC8', 'PTC9', 'data_point', 'recipes',
       'autoclave', 'date', 'number', 'PTC_count'],
      dtype='object')

In [18]:
all_data['TemParameter'] = 0

In [19]:
G17_data = pd.DataFrame()

In [24]:
for PTC in PTC_list:
    data = all_data.loc[~all_data[PTC].isnull(),
             ['number', 'date', 'autoclave', 'recipes'
              ,'TemParameter', 'PTC_count','data_point',PTC]] #將各別 PTC 非0的數據挑出
    data['TemParameter'] = PTC  #建立 TemParameter 對應所屬來的PTC
    data['value'] = data[PTC]  # 賦予value值 統一
    data.drop([PTC],inplace=True,axis = 1)
    G17_data = pd.concat([G17_data,data])

In [25]:
G17_data.reset_index(drop=True, inplace=True)

In [28]:
G17_data['TemParameter'].value_counts()

PTC15    4023
PTC16    4012
PTC10    4011
PTC14    3998
PTC13    3690
PTC9     3382
PTC11    3092
PTC17    3078
PTC12    2771
PTC7     2420
PTC8      953
PTC18     947
PTC5      305
Name: TemParameter, dtype: int64

In [29]:
G17_data.to_csv('./training_data/G17_data.csv')